In [ ]:
import sqlite3
import os
import pysolr

sqlitedb = os.path.join(os.path.expanduser('~'),'Box Sync', 'GradSchoolStuff', 'MastersProject', 'mimic3', 'mimic3.sqlite')
if not (os.path.exists(sqlitedb)):
    print("Specified database does not exist")
    sys.exit()

connection = sqlite3.connect(sqlitedb)

# Connect to Solr instance.
solr = pysolr.Solr('http://192.168.99.100:8983/solr/radiology')

with connection:
    cur = connection.cursor()
    recordset = cur.execute("""
        SELECT * FROM noteevents
        WHERE description LIKE '%CT%'
            AND category = 'Radiology'
        ORDER BY row_id""")
    col_names = [cn[0] for cn in cur.description]

    for row in recordset:
        document = {}
        #print('++++++', col_names[idx], ': ', col)
        # want row_id, subject_id, hadm_id, description, text
        document['id'] = row[col_names.index("ROW_ID")]
        document['SUBJECT_ID_i'] = row[col_names.index("SUBJECT_ID")]
        # not all reports have a hospital admission id, so this may not be in Solr
        document['HADM_ID_i'] = row[col_names.index("HADM_ID")]
        # store description as a single string, but also allow for searching parts
        document['DESCRIPTION_s'] = row[col_names.index("DESCRIPTION")]
        document['DESCRIPTION_t'] = row[col_names.index("DESCRIPTION")]
        document['TEXT_t'] = row[col_names.index("TEXT")]
        
        """
        Out of 108,896 reports, only 4 don't use 'REASON FOR THIS EXAMINATION:' 
        or 'Reason:' for the reason for exam section. By manual inspection, those 4
        reports are not ones related to PE nor PAH.
        
        Reports can have multiple reason sections. From inspection of the data,
          * ^'Reason:' (if it exists) is just a one line section and is a portion of the 
            'REASON FOR THIS EXAMINATION:' section (if it exists)
          * ^'REASON FOR THIS EXAMINATION:' (if it exists) is a multi-line section and the actual
            reason starts on the next line. There are 3 cases where this section was repeated and
            in some of the repeated locations the reason may partially be on the same line.
        
        """
        # need to change these if x in to regex to make sure we're looking at the section that starts a new row.
        if 'REASON FOR THIS EXAMINATION:' in row[col_names.index("TEXT")] or 'Reason:' in row[col_names.index("TEXT")]:
            found = ''
            for line in row[col_names.index("TEXT")].rstrip().split('\n'):
                #if 'REASON FOR THIS EXAMINATION:' in line or 'Reason:' in line:
                #if 'Reason:' in line:
                if 'REASON FOR THIS EXAMINATION:' in line:
                    test = line.replace('REASON FOR THIS EXAMINATION:', '').rstrip()
                    if len(test) > 0:
                        print(row[col_names.index("ROW_ID")], 'line:', line)
                    found = True
                    #print(row[col_names.index("ROW_ID")], 'line:', line)
                elif found:
                    #if ':' not in line and '___________________' not in line and '**************' not in line:
                    #print(row[col_names.index("ROW_ID")], 'next line:', line)
                    found = False
            #print('found reason')
            #print('Adding row_id', row[col_names.index("ROW_ID")], 'to Solr.')
            #solr.add([document])
        else:
            print('------------missing for', row[col_names.index("ROW_ID")])


In [ ]:
#Now need to devide reports into sections...
# Or should this be done before importing into Solr???

# Right now I think that section segmentation should be done before solr insertion as the section
# can be inserted as another data field.
# So far, Reason and Impression sections are those that I think most relevant for looking for PE/PAH
# of course need to see how many reports actually have an impressions section.